In [20]:
import pandas as pd
from sklearn.model_selection import KFold, train_test_split
from lohrasb.best_estimator import BaseModel
from sklearn.pipeline import Pipeline
from feature_engine.imputation import (
    CategoricalImputer,
    MeanMedianImputer
    )
from category_encoders import OrdinalEncoder
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    f1_score)
from sklearn.metrics import f1_score, accuracy_score, make_scorer
from lightgbm import *


# Example: Use Adult Data Set (a classification problem)
  
https://archive.ics.uci.edu/ml/datasets/Adult

# Part 1: Use BestModel in sklearn pipeline


In [21]:
urldata= "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
# column names
col_names=["age", "workclass", "fnlwgt" , "education" ,"education-num",
"marital-status","occupation","relationship","race","sex","capital-gain","capital-loss","hours-per-week",
"native-country","label"
]
# read data
data = pd.read_csv(urldata,header=None,names=col_names,sep=',')
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


#### Define labels


In [22]:
data.loc[data['label']=='<=50K','label']=0
data.loc[data['label']==' <=50K','label']=0

data.loc[data['label']=='>50K','label']=1
data.loc[data['label']==' >50K','label']=1

data['label']=data['label'].astype(int)

#### Train test split

In [23]:
X = data.loc[:, data.columns != "label"]
y = data.loc[:, data.columns == "label"]


X_train, X_test, y_train, y_test =train_test_split(X, y, \
     test_size=0.33, stratify=y['label'], random_state=42)


#### Find feature types for later use

In [24]:
int_cols =  X_train.select_dtypes(include=['int']).columns.tolist()
float_cols =  X_train.select_dtypes(include=['float']).columns.tolist()
cat_cols =  X_train.select_dtypes(include=['object']).columns.tolist()


#### Define estimator and set its arguments 


In [25]:
estimator = LGBMClassifier()
estimator_params = {
        "max_depth": [6,30],
        "learning_rate": [0.001,0.01, 0.1],
        "n_estimators": [100,200,1000],
        "boosting_type": ["gbdt"],

    }
    

In [26]:

kwargs = {  # params for fit method or fit_params 
            'fit_random_kwargs' :{
            'sample_weight':None,
            },
            # params for RandomSearchCV 
            'random_search_kwargs' : {
            'estimator':estimator,
            'param_distributions':estimator_params,
            'scoring' :'f1',
            'verbose':3,
            'n_jobs':-1,
            'cv':KFold(2),
            'n_iter':10,
            'refit':True,
            }
            }



In [27]:

obj = BaseModel().optimize_by_randomsearchcv(
    kwargs=kwargs
 )

#### Build sklearn pipeline

In [28]:


pipeline =Pipeline([
            # int missing values imputers
            ('intimputer', MeanMedianImputer(
                imputation_method='median', variables=int_cols)),
            # category missing values imputers
            ('catimputer', CategoricalImputer(variables=cat_cols)),
            #
            ('catencoder', OrdinalEncoder()),
            # classification model
            ('obj', obj)

 ])


#### Run Pipeline

In [29]:
pipeline.fit(X_train,y_train.values.ravel())
y_pred = pipeline.predict(X_test)


Fitting 2 folds for each of 10 candidates, totalling 20 fits
[CV 1/2] END boosting_type=gbdt, learning_rate=0.1, max_depth=30, n_estimators=100;, score=0.696 total time=   0.5s
[CV 2/2] END boosting_type=gbdt, learning_rate=0.1, max_depth=30, n_estimators=100;, score=0.705 total time=   0.5s
[CV 1/2] END boosting_type=gbdt, learning_rate=0.01, max_depth=6, n_estimators=100;, score=0.604 total time=   0.5s
[CV 2/2] END boosting_type=gbdt, learning_rate=0.01, max_depth=6, n_estimators=100;, score=0.542 total time=   0.5s
[CV 1/2] END boosting_type=gbdt, learning_rate=0.01, max_depth=6, n_estimators=200;, score=0.657 total time=   0.9s
[CV 2/2] END boosting_type=gbdt, learning_rate=0.01, max_depth=6, n_estimators=200;, score=0.648 total time=   0.9s
[CV 1/2] END boosting_type=gbdt, learning_rate=0.1, max_depth=6, n_estimators=200;, score=0.693 total time=   0.8s
[CV 2/2] END boosting_type=gbdt, learning_rate=0.1, max_depth=6, n_estimators=200;, score=0.705 total time=   0.8s
[CV 1/2] END 

#### Check performance of the pipeline

In [30]:
print('F1 score : ')
print(f1_score(y_test,y_pred))
print('Classification report : ')
print(classification_report(y_test,y_pred))
print('Confusion matrix : ')
print(confusion_matrix(y_test,y_pred))


F1 score : 
0.7156348373557189
Classification report : 
              precision    recall  f1-score   support

           0       0.90      0.94      0.92      8158
           1       0.78      0.66      0.72      2588

    accuracy                           0.87     10746
   macro avg       0.84      0.80      0.82     10746
weighted avg       0.87      0.87      0.87     10746

Confusion matrix : 
[[7686  472]
 [ 883 1705]]


# Part 2:  Use BestModel as a standalone estimator 

In [31]:
X_train, X_test, y_train, y_test =train_test_split(X, y, \
     test_size=0.33, stratify=y['label'], random_state=42)

#### Transform features to make them ready for model input

In [32]:
transform_pipeline =Pipeline([
            # int missing values imputers
            ('intimputer', MeanMedianImputer(
                imputation_method='median', variables=int_cols)),
            # category missing values imputers
            ('catimputer', CategoricalImputer(variables=cat_cols)),
            #
            ('catencoder', OrdinalEncoder()),
            # classification model

 ])

#### Transform X_train and X_test

In [33]:
X_train=transform_pipeline.fit_transform(X_train,y_train)
X_test=transform_pipeline.transform(X_test)


#### Train model and predict

In [34]:
obj.fit(X_train,y_train.values.ravel())
y_pred = obj.predict(X_test)

Fitting 2 folds for each of 10 candidates, totalling 20 fits
[CV 2/2] END boosting_type=gbdt, learning_rate=0.001, max_depth=6, n_estimators=100;, score=0.000 total time=   0.3s
[CV 1/2] END boosting_type=gbdt, learning_rate=0.001, max_depth=6, n_estimators=100;, score=0.000 total time=   0.3s
[CV 1/2] END boosting_type=gbdt, learning_rate=0.1, max_depth=30, n_estimators=100;, score=0.696 total time=   0.4s
[CV 2/2] END boosting_type=gbdt, learning_rate=0.1, max_depth=30, n_estimators=100;, score=0.705 total time=   0.4s
[CV 1/2] END boosting_type=gbdt, learning_rate=0.001, max_depth=30, n_estimators=200;, score=0.000 total time=   0.7s
[CV 2/2] END boosting_type=gbdt, learning_rate=0.001, max_depth=30, n_estimators=200;, score=0.000 total time=   0.8s
[CV 1/2] END boosting_type=gbdt, learning_rate=0.1, max_depth=6, n_estimators=100;, score=0.696 total time=   0.3s
[CV 2/2] END boosting_type=gbdt, learning_rate=0.1, max_depth=6, n_estimators=100;, score=0.703 total time=   0.3s
[CV 1/2

#### Check performance of the model

In [39]:
print('accuracy score score : ')
print(accuracy_score(y_test,y_pred))
print('F1 score : ')
print(f1_score(y_test,y_pred))
print('Classification report : ')
print(classification_report(y_test,y_pred))
print('Confusion matrix : ')
print(confusion_matrix(y_test,y_pred))

accuracy score score : 
0.8739065698864694
F1 score : 
0.7156348373557189
Classification report : 
              precision    recall  f1-score   support

           0       0.90      0.94      0.92      8158
           1       0.78      0.66      0.72      2588

    accuracy                           0.87     10746
   macro avg       0.84      0.80      0.82     10746
weighted avg       0.87      0.87      0.87     10746

Confusion matrix : 
[[7686  472]
 [ 883 1705]]


In [36]:
obj.get_best_estimator()

LGBMClassifier(learning_rate=0.01, max_depth=30, n_estimators=1000)

In [37]:
obj.best_estimator

LGBMClassifier(learning_rate=0.01, max_depth=30, n_estimators=1000)

#### Get fitted randomized search object and its attributes

In [38]:
RandomSearchObj = obj.get_optimized_object()
RandomSearchObj.cv_results_

{'mean_fit_time': array([1.82551789, 2.05567443, 0.221367  , 2.28969336, 0.37693715,
        0.21559751, 0.25415099, 2.47853899, 0.48186064, 1.38753903]),
 'std_fit_time': array([4.10239697e-02, 3.00055742e-02, 4.86600399e-03, 1.76055431e-02,
        6.44421577e-03, 6.43742085e-03, 1.29938126e-05, 4.37428951e-02,
        8.08143616e-03, 1.11945033e-01]),
 'mean_score_time': array([3.0091635 , 4.86560953, 0.09694052, 3.37929499, 0.26257241,
        0.13971031, 0.09829402, 2.51973152, 0.3063699 , 0.87099695]),
 'std_score_time': array([0.21900856, 0.0205735 , 0.00110555, 0.02846301, 0.0015825 ,
        0.00276244, 0.00438595, 0.03373146, 0.02599418, 0.00338602]),
 'param_n_estimators': masked_array(data=[1000, 1000, 100, 1000, 200, 100, 100, 1000, 200, 1000],
              mask=[False, False, False, False, False, False, False, False,
                    False, False],
        fill_value='?',
             dtype=object),
 'param_max_depth': masked_array(data=[6, 30, 6, 30, 30, 30, 30, 6, 6